In [1]:
!pip install pyvis

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyvis.network import Network
from pyvis import network as net
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import IPython

# Visualización de la Información - Tarea 13
Elaborada por: __Andrés Urbano Guillermo Gerardo__

_18 de Mayo del 2022_

Cargamos nuestro conjunto de datos.

In [3]:
df = pd.read_csv("World_Vaccination_Progress.csv")
df.tail()

,Country,Doses Administered,Doses per 1000,Fully Vaccinated Population (%),Vaccine being used in a country
173,Anguilla,13235,732.6,24.5,Oxford/AstraZeneca
174,Wallis and Futuna,8834,558.8,27.2,Oxford/AstraZeneca
175,Bonaire Sint Eustatius and Saba,7391,275.7,6.2,"Moderna, Pfizer/BioNTech"
176,Falkland Islands,4322,1070.1,42.5,Oxford/AstraZeneca
177,Montserrat,1306,243.7,3.6,Oxford/AstraZeneca


Para generar el modelo de red con PyVis seleccionaremos las vacunas que son usadas por cada pais en un nuevo dataframe.

In [4]:
countries = df['Country'][1:].values
list_vaccines =  df['Vaccine being used in a country'][1:].values

data_countries = []
data_vaccines = []
for country, vaccines in zip(countries, list_vaccines):
    list_vaccines_string = vaccines.split(',')
    data_countries.extend([country for vaccine in list_vaccines_string])
    data_vaccines.extend(list_vaccines_string)    

vaccine_df = pd.DataFrame(dict(
    Source=data_countries,
    Target=data_vaccines    
))
vaccine_df.head()

,Source,Target
0,United States,Johnson&Johnson
1,United States,Moderna
2,United States,Pfizer/BioNTech
3,India,Covaxin
4,India,Oxford/AstraZeneca


In [10]:
# Generamos nuestra gráfica con PyVis
G=nx.from_pandas_edgelist(vaccine_df, 'Source', 'Target')
vaccine_net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black', notebook=True)
vaccine_net.from_nx(G)
vaccine_net.show("Vaccine_net.html")
#IPython.display.HTML(filename='Vaccine_net.html')

## Segunda forma de visualizar

Otra manera de visualizar nuestra red será coloreando con diferentes colores los nodos, de esta manera reconocer los cluster que generan cuando se crean las relaciones.

In [8]:
world = df.iloc[0].Country
root = [world]

main_cluster = list(vaccine_df.Source.unique())

sub_cluster = list(vaccine_df.Target.unique())

data = ([list(relacion) for relacion in vaccine_df.values])

In [11]:
"""Define edges."""
from more_itertools import locate

test_nw = Network(height='750px', width="100%", notebook=True)
nodes = root + main_cluster + sub_cluster

# añadir nodo raíz
root_node = list(locate(nodes, lambda x: x in root))
root_size, root_color = [35 for _ in root], ["#006e7f" for _ in root]

test_nw.add_nodes(root_node, size=root_size, label=root, color=root_color)

# agregar nodos cluster principal
main_nodes = list(locate(nodes, lambda x: x in main_cluster))
main_size, main_color = [30 for _ in main_cluster], ["#F8CB2E" for _ in main_cluster]

test_nw.add_nodes(main_nodes, size=main_size, label=main_cluster, color=main_color)

# agregar nodos de subclúster
sub_nodes = list(locate(nodes, lambda x: x in sub_cluster))
sub_size, sub_color = [25 for _ in sub_cluster], ["#EE5007" for _ in sub_cluster]

test_nw.add_nodes(sub_nodes, size=sub_size, label=sub_cluster, color=sub_color)

# agregar bordes
for edge in data:
    node_from, node_to = list(locate(nodes, lambda x: x in edge))
    test_nw.add_edge(node_from, node_to)

test_nw.show("test.html")
#IPython.display.HTML(filename='test.html')